In [1]:
import subprocess as sp
import networkx as nx
import matplotlib.pyplot as plt


import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import neuraltda.simpComp as sc
import numpy as np
import tempfile
import h5py as h5
import glob
import os
from importlib import reload
import tqdm
import pickle
%matplotlib inline


import datetime
daystr = datetime.datetime.now().strftime('%Y%m%d')
figsavepth = '/home/brad/DailyLog/'+daystr+'/'
print(figsavepth)

/home/brad/.conda/envs/neuraltda-4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/brad/DailyLog/20181110/


In [2]:
# Set up birds and block_paths
birds = ['B1083', 'B1056', 'B1235', 'B1075']
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'}

bird = 'B1083'
block_path = bps[bird]

# Parameters for binning
winsize = 10.0               # binning window size in milliseconds
dtovr = winsize*0.5          # window overlap in milliseconds
comment = 'ForSCM'         # We're plotting betti curves
segment_info = [0,0]       # We want the whole stimulus and just the stimulus

# Parameters for Betti curve computation
thresh = 13.0                            # Activation threshold
nperms = None                            # No permutations of the population - use full pop
ncellsperm = 20                          # Number of cells to subsample for each permuation
dims = [0,1,2,3,4]                        # Compute first five betti numbers
Ntimes = 1000                            # Number of interpolation points
stimlen = 6000                           # Stimulus length in milliseconds
twin = np.linspace(0, stimlen, Ntimes)   # Vector of interpolation time points


In [3]:
# Bin the data from each bird
bdfs = {}
for bird in birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, winsize, segment_info, cluster_group=['Good'], dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs[bird] = bdf

/home/brad/krista/B1083/P03S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForSCM/20170829T191155Z-10.0-5.0.binned
/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForSCM/20180612T185704Z-10.0-5.0.binned
/home/brad/krista/B1235/P02S01/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForSCM/20180612T220619Z-10.0-5.0.binned
/home/brad/krista/B1075/P01S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForSCM/20180612T222949Z-10.0-5.0.binned


In [5]:
reload(tp2)
bc_tavg_store = {}
t_store = {}
tms_store = {}
bc_tavg_shuff_store = {}

for bird in birds:
    bdf = bdfs[bird]
    block_path = bps[bird]
    # compute real and shuffled betti curves - TrialAverage
    print('Computing trial average')
    (bc_tavg, t, tms) = tp2.compute_trialaverage_betti_curves('ForSCMControl', block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
    (bc_tavg_shuff, t, tms) = tp2.compute_trialaverage_betti_curves('ForSCMControl-Shuffle', block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr, shuffle=True)
    bc_tavg_store[bird] = bc_tavg
    bc_tavg_shuff_store[bird] = bc_tavg_shuff
    t_store[bird] = t
    tms_store[bird] = tms

Computing trial average
Computing trial average
Computing trial average
Computing trial average


In [ ]:
def plot_scm_distribution(scm_bettis, trial, bettinum, maxbin=200, ax=None, **kwargs):
    nmaxtot = 0
    scm_trial_bettis = scm_bettis[trial]
    bettivals = []
    try:
        for x in scm_trial_bettis:
            try:
                bettivals.append(x[0][bettinum])
            except IndexError:
                bettivals.append(0)
    except IndexError:
        bettivals = [x[bettinum] for x in scm_trial_bettis]
    bettivals2 = []
    for x in bettivals:
        if type(x) is not list:
            bettivals2.append(x)
        else:
            bettivals2.append(x[0])
    #print(bettivals)
    maxbettival = np.amax(bettivals2)
    maxbin = round(1.5*maxbettival)
    print(maxbin)
    bins = np.arange(maxbin)
    (n, bins2, ptaches) = ax.hist(bettivals2, bins, normed=True, stacked=True, **kwargs)
    #print(n)
    nmax = np.amax(n)
    nmaxtot = np.amax([nmax, nmaxtot])
    nmaxx = np.amax(bins)
    return (nmaxtot, nmaxx)

def plot_scm_distrib(scm_bettis, bettinum, maxbin=200, ax=None, **kwargs):
    
    nmaxtot = 0
    
    bettis = scm_bettis[:, 0, bettinum]
    bettis = bettis[bettis >=0]
    #print(bettis)
    maxbettival = np.amax(bettis)
    maxbin = round(1.5*(maxbettival+1))
    bins = np.arange(maxbin)
    (n, bins2, ptaches) = ax.hist(bettis, bins, normed=True, stacked=True, **kwargs)
    #print(n)
    nmax = np.amax(n)
    nmaxtot = np.amax([nmax, nmaxtot])
    nmaxx = np.amax(bins)
    return (nmaxtot, nmaxx)

In [ ]:
# average over all trials
import tqdm
nsamples = 200
bird_scm_dict = {}
for bird in birds:
    bdf = bdfs[bird]
    stim_dict = {}
    stim_shuffled_dict = {}
    stim_scm_dict = {}
    stim_betticurve_dict = {}
    stim_shuffled_betticurve_dict = {}
    with h5.File(bdf, 'r') as bdff:
        stims = bdff.keys()
        for stim in tqdm.tqdm(stims):
            #print(stim)
            trial=0
            stim_poptens = tp2.extract_population_tensor(bdf, stim)
            ntrials = tp2.num_trials(stim_poptens)
            stim_scm_dat= []
            stim_trialavg_poptens = np.mean(stim_poptens, axis=2)[:, :, np.newaxis]     
            scm_bettis = tp2.calc_scm_betti_distribution(stim_trialavg_poptens, thresh, trial, nsamples)
            stim_scm_dict[stim] = scm_bettis
    bird_scm_dict[bird] = stim_scm_dict


  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Extracting Population Activity Tensor...
Stim: I_40k, Clusters:None


In [ ]:
import pickle
with open(os.path.join(figsavepth,'scm_control_save_trialvg_ALL.pkl'), 'wb') as f:
    pickle.dump(bird_scm_dict, f)

In [ ]:
plt.style.use('/home/brad/code/NeuralTDA/gentnerlab.mplstyle')
f, axs = plt.subplots(2, 8,figsize=(20, 4), sharey=True, sharex=False)
betti_num = 3
bettis = [1,2]
stims = list(bc_tavg.keys())[0:8]
my = 0
for ind, betti_num in enumerate(bettis):
    nmaxyt = 0
    nmaxxt = 0
    for stimind, stim in enumerate(stims):
        final_bettis = bc_tavg[stim][:, -1, 0]
        ax = axs[ind, stimind]
        #final_bettis = stim_betticurve_dict[stim][-1]
        b = final_bettis[betti_num]
        ax.plot([b, b], [0, 100], 'k--', linewidth=2)
        (nmaxy, nmaxx) = plot_scm_distribution([stim_scm_dict[stim]], 0, betti_num, ax=ax, align='left', histtype='stepfilled')
        nmaxyt = np.amax([nmaxy, nmaxyt])
        nmaxxt = np.amax([nmaxx, nmaxxt])
        print(final_bettis)

        if ind > 0:
            xmax = 250
            
            ax.set_xticks(np.arange(0, 300, 100))
        else:
            xmax = 100
    
        ax.set_xlim([-0.1, xmax])
        ax.set_ylim([0, 0.12])
        ax.tick_params(labelsize=12)
        for i in ax.get_xticklabels():
            i.set_fontname('helvetica')
        for i in ax.get_yticklabels():
            i.set_fontname('helvetica')
        for axis in ['top', 'bottom', 'left', 'right']:
            ax.spines[axis].set_linewidth(2)
            
        
        #ax.set_title(stim)
plt.savefig(os.path.join(figsavepth, 'SCMControl{}_bettiAll_8stims.eps'.format(bird)))